In [1]:
path = '../blockchain_files/3000000to3999999_BlockTransaction/'

In [2]:
import pandas as pd
from datetime import datetime
import time
import numpy as np
import torch
from torch_geometric.data import Data

from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset

In [3]:
df = pd.read_csv(path+'final.csv')

In [4]:
labels = df['0'].unique()
for label in labels:
    print('%s: '% label,(df['0'] == label).sum())
# print('mining: ',(merged_df['0'] == 'mining').sum())

mining:  3499995
nan:  0
exchange:  1315278
walletapp:  52765
compromised:  24937
gambling:  823
isowallet:  135
phish_hack:  172


In [5]:
print(labels[2])
exchange_data = df[df['0'] == labels[2]]
exchange_data.head()

exchange


,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
22,1484475182,13,1676,0.989400,33.3333,3.0,2.1,exchange
26,1484475182,13,3522,2500.000000,33.3333,3.0,2.1,exchange
37,1484475230,17,54938,25.237936,4.0000,6.0,2.1,exchange
80,1484475418,33,1890243,0.010000,9.0000,2.0,2.1,exchange
84,1484475449,17,55130,0.322861,4.0000,6.0,2.1,exchange


In [6]:
exchange_addrs = exchange_data['from'].unique()

In [7]:
kept_ex_addrs = []
for addr in exchange_addrs:
    if (exchange_data['from']==addr).sum() > 100:
        kept_ex_addrs.append(addr)
#     print( str(addr)+' :',(exchange_data['from']==addr).sum())
    

In [8]:
print(labels[0])
mining_data = df[df['0'] == labels[0]]
mining_data.head()

mining


,timestamp,from,to,value,gasLimit,gasPrice,gasUsed,0
0,1484475035,0,1890222,4.950000,3.9,2.0,2.1,mining
1,1484475035,1,1777,1.011962,9.0,2.0,2.1,mining
2,1484475035,1,20425,1.011957,9.0,2.0,2.1,mining
3,1484475035,1,1558,1.008363,9.0,2.0,2.1,mining
4,1484475035,1,15,1.016120,9.0,2.0,2.1,mining


In [9]:
mining_addrs = mining_data['from'].unique()

In [10]:
kept_mining_addrs = []
for addr in mining_addrs:
    if (mining_data['from']==addr).sum() > 100:
        kept_mining_addrs.append(addr)
#     print( str(addr)+' :',(mining_data['from']==addr).sum())
    

In [11]:
def receiving_transaction_graph_h4_timed(node,st_time,end_time,dataframe):
    
    timed_df = dataframe[dataframe['timestamp'].between(st_time,end_time)]
    
    neigh1 = timed_df[timed_df['to'] == node]
    
    neigh2 = timed_df[timed_df['to'].isin(neigh1['from'])]
    neigh2=neigh2[neigh2['to'] != node]
    
    neigh3 = timed_df[timed_df['to'].isin(neigh2['from'])]
    neigh3 = neigh3[~neigh3['to'].isin(neigh2['to'])]
    
    neigh4 = timed_df[timed_df['to'].isin(neigh3['from'])]
    neigh4 = neigh4[~neigh4['to'].isin(neigh3['to'])]
    
    return (neigh1,neigh2,neigh3,neigh4)
    
    

In [12]:
def send_extract_graph_h4_timed(node,st_time,end_time,dataframe):
    
    timed_df = dataframe[dataframe['timestamp'].between(st_time,end_time)]
    
    neigh1 = timed_df[timed_df['from'] == node]
    
    neigh2 = timed_df[timed_df['from'].isin(neigh1['to'])]
    neigh2=neigh2[neigh2['from'] != node]
    
    neigh3 = timed_df[timed_df['from'].isin(neigh2['to'])]
    neigh3 = neigh3[~neigh3['from'].isin(neigh2['from'])]
    
    neigh4 = timed_df[timed_df['from'].isin(neigh3['to'])]
    neigh4 = neigh4[~neigh4['from'].isin(neigh3['from'])]
    
    return (neigh1,neigh2,neigh3,neigh4)
    
    


In [13]:
def aggregate_tx(data):
    selected_data = data[['from','to','value','gasLimit','gasPrice','gasUsed']].reset_index(drop=True)
    grouped_data = selected_data.groupby(['from','to'])
    
    agg_counts = grouped_data.size()
    agg_neigh = grouped_data.mean()
    
    agg_neigh['counts'] = agg_counts
    
    return agg_neigh.reset_index()


In [24]:
def get_aggregated(graph):
    
    (neigh1,neigh2,neigh3,neigh4) = graph
    
    neigh1 = aggregate_tx(neigh1)
    neigh2 = aggregate_tx(neigh2)
    neigh3 = aggregate_tx(neigh3)
    neigh4 = aggregate_tx(neigh4)
    
    return (neigh1,neigh2,neigh3,neigh4)

def combine(send_graph,rec_graph):
    
    (neigh1_send,neigh2_send,neigh3_send,neigh4_send) = send_graph
    (neigh1_rec,neigh2_rec,neigh3_rec,neigh4_rec) = rec_graph
    
    
    neigh1 = pd.concat([neigh1_send,neigh1_rec])
    neigh2 = pd.concat([neigh2_send,neigh2_rec])
    neigh3 = pd.concat([neigh3_send,neigh3_rec])
    neigh4 = pd.concat([neigh4_send,neigh4_rec])
    
    
    return (neigh1,neigh2,neigh3,neigh4)

st_time = 1484481572
duration = 700000
end_time = st_time+duration
addr = kept_ex_addrs[1]
    
send_graph = send_extract_graph_h4_timed(addr,st_time,end_time,df)
rec_graph = receiving_transaction_graph_h4_timed(addr,st_time,end_time,df)


# (neigh1,neigh2,neigh3,neigh4) = get_aggregated(rec_graph)

(neigh1,neigh2,neigh3,neigh4) = combine(send_graph,rec_graph)

In [25]:
def extract_graphs(addr,duration):
    st_time = df.iloc[0]['timestamp']
#     duration = 700000
#     addr = 261
    end_time = 0
    graphs = []
    while  end_time < df.iloc[-1]['timestamp']:
        end_time = st_time+duration
        
        send_graph = send_extract_graph_h4_timed(addr,st_time,end_time,df)
        rec_graph = receiving_transaction_graph_h4_timed(addr,st_time,end_time,df)
        
        (neigh1,neigh2,neigh3,neigh4) = combine(send_graph,rec_graph)
        graphs.append((neigh1,neigh2,neigh3,neigh4))
        st_time = end_time

        
    return graphs

In [26]:
miner_graph = []
duration = 700000
for addr in kept_mining_addrs:
    miner_graph.append(extract_graphs(addr,duration))

In [27]:
ex_graph = []
duration = 700000
for addr in kept_ex_addrs:
    ex_graph.append(extract_graphs(addr,duration))

In [50]:
def get_graph_data(graph,label):
    (neigh1,neigh2,neigh3,neigh4) = graph
    neigh1_nodes = pd.concat([neigh1['from'],neigh1['to']])
    neigh2_nodes = pd.concat([neigh2['from'],neigh2['to']])
    neigh3_nodes = pd.concat([neigh3['from'],neigh3['to']])
    neigh4_nodes = pd.concat([neigh4['from'],neigh4['to']])

    nodes = pd.concat([neigh1_nodes
                  ,neigh2_nodes
                  ,neigh3_nodes
                  ,neigh4_nodes]).unique()
    map_id = {j:i for i,j in enumerate(nodes)}
    
    neigh1['from'] = neigh1['from'].map(map_id)
    neigh1['to'] = neigh1['to'].map(map_id)
    
    neigh2['from'] = neigh2['from'].map(map_id)
    neigh2['to'] = neigh2['to'].map(map_id)
    
    neigh3['from'] = neigh3['from'].map(map_id)
    neigh3['to'] = neigh3['to'].map(map_id)
    
    neigh4['from'] = neigh4['from'].map(map_id)
    neigh4['to'] = neigh4['to'].map(map_id)
    
    graph = pd.concat([neigh1,neigh2,neigh3,neigh4])
    
    edge_index = torch.tensor(graph[['from','to']].to_numpy().T, dtype=torch.long)
    
    x = torch.ones(nodes.shape).reshape(-1,1)
    edge_attributes = torch.tensor(graph[['value','gasLimit','gasPrice','gasUsed' ]].to_numpy())
    
#     print(label)
    data = Data(x=x, edge_index=edge_index,edge_attr = edge_attributes,y=torch.tensor([label]))
    
#     print(edge_attributes.shape)

    
    neigh1 = neigh1.sort_values(['from','to','timestamp'])
    neigh2 = neigh2.sort_values(['from','to','timestamp'])
    neigh3 = neigh3.sort_values(['from','to','timestamp'])
    neigh4 = neigh4.sort_values(['from','to','timestamp'])
    
    neigh1 = neigh1.reset_index(drop=True)
    neigh1_gb = neigh1.groupby(['from','to'])
    neigh2 = neigh2.reset_index(drop=True)
    neigh2_gb = neigh2.groupby(['from','to'])
    neigh3 = neigh3.reset_index(drop=True)
    neigh3_gb = neigh3.groupby(['from','to'])
    neigh4 = neigh4.reset_index(drop=True)
    neigh4_gb = neigh4.groupby(['from','to'])
    
    groups = neigh1_gb.groups
    keys = list(groups.keys())
    print(len(keys))
#     print(neigh1.loc[groups[keys[0]]])
    
    groups = neigh2_gb.groups
    keys = list(groups.keys())
    print(len(keys))
    
    
    groups = neigh3_gb.groups
    keys = list(groups.keys())
    print(len(keys))
    
    groups = neigh4_gb.groups
    keys = list(groups.keys())
    print(len(keys))
    
    return data
data = get_graph_data(ex_graph[3][0],0)
data

268
223
2087
2812


Data(x=[3735, 1], edge_index=[2, 12726], edge_attr=[12726, 4], y=[1])

In [23]:
1542+4614+6153+4718

17027

In [47]:
953+1250+2064+2399

6666